# This notebook explore the trainline API as constructed here : https://github.com/tducret/trainline-python
## It downloads all available stops and allows the user to call the trainline API and get the results in a readable form

In [58]:
import pandas as pd
import requests
import io 
import json 
import copy
from datetime import datetime as dt

pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)


In [59]:
# Custom function to handle DF
def pandas_explode(df, column_to_explode):
    """
    Similar to Hive's EXPLODE function, take a column with iterable elements, and flatten the iterable to one element 
    per observation in the output table

    :param df: A dataframe to explod
    :type df: pandas.DataFrame
    :param column_to_explode: 
    :type column_to_explode: str
    :return: An exploded data frame
    :rtype: pandas.DataFrame
    """

    # Create a list of new observations
    new_observations = list()

    # Iterate through existing observations
    for row in df.to_dict(orient='records'):

        # Take out the exploding iterable
        explode_values = row[column_to_explode]
        del row[column_to_explode]

        # Create a new observation for every entry in the exploding iterable & add all of the other columns
        for explode_value in explode_values:

            # Deep copy existing observation
            new_observation = copy.deepcopy(row)

            # Add one (newly flattened) value from exploding iterable
            new_observation[column_to_explode] = explode_value

            # Add to the list of new observations
            new_observations.append(new_observation)

    # Create a DataFrame
    return_df = pd.DataFrame(new_observations)

    # Return
    return return_df


In [22]:
# Get all train and bus station from trainline thanks to https://github.com/tducret/trainline-python
_STATIONS_CSV_FILE = "https://raw.githubusercontent.com/\
trainline-eu/stations/master/stations.csv"

def update_trainline_stops(url = _STATIONS_CSV_FILE):

    csv_content = requests.get(url).content
    all_stops = pd.read_csv(io.StringIO(csv_content.decode('utf-8')),
                     sep=';', index_col=0, low_memory=False)
    # filter on station with parnt_station_id (that we can call the API with)
    all_stops = all_stops[~pd.isna(all_stops.parent_station_id)]
    # Group info on bus or train
    all_stops['is_bus_station']= all_stops.apply(lambda x: (x.busbud_is_enabled =='t')or(x.flixbus_is_enabled == 't'), axis=1)
    all_stops['is_train_station']= all_stops.apply(lambda x: (x.sncf_is_enabled =='t')or(x.idtgv_is_enabled == 't')
                                                   or(x.db_is_enabled == 't')or(x.cff_is_enabled == 't')
                                                   or(x.leoexpress_is_enabled == 't')or(x.obb_is_enabled == 't')
                                                   or(x.ntv_is_enabled == 't')or(x.hkx_is_enabled == 't')
                                                   or(x.renfe_is_enabled == 't')or(x.atoc_is_enabled == 't')
                                                   or(x.benerail_is_enabled == 't')or(x.westbahn_is_enabled == 't')
                                                   or(x.ouigo_is_enabled == 't')or(x.trenitalia_is_enabled == 't'), axis=1)
    all_stops['geoloc'] = all_stops.apply(lambda x: [x.latitude,x.longitude], axis=1)
    # Keep only relevant columns
    all_stops = all_stops[['name', 'slug', 'country', 'latitude','longitude','geoloc', 'parent_station_id'
                           ,'is_bus_station','is_train_station']]
    print(f'{all_stops.shape[0]} stops were found. Here is an example:\n {all_stops.sample()}')
    return all_stops

all_stops = update_trainline_stops()


2821 were found. Here is an example:
                       name                  slug country   latitude  \
id                                                                    
1332  Casino—Lacroix-Laval  casino-lacroix-laval      FR  45.787811   

      longitude                   geoloc  parent_station_id  is_bus_station  \
id                                                                            
1332   4.730938  [45.7878115, 4.7309383]             4862.0           False   

      is_train_station  
id                      
1332              True  


In [75]:
passengers = [{'id': '3c29a998-270e-416b-83f0-936b606638da', 'age': 39,
               'cards': [], 'label': '3c29a998-270e-416b-83f0-936b606638da'}]

# Fucntion to format the trainline json repsonse
def format_trainline_response(rep_json, segment_details = False, only_sellable = True):
    """ 
    Format complicated json with information flighing around into a clear dataframe
    """
    # get folders (aggregated outbound or inbound trip)
    folders = pd.DataFrame.from_dict(rep_json['folders'])
    #print(f'we got {legs.shape[0]} legs')

    folders['nb_segments'] = folders.apply(lambda x: len(x['trip_ids']),axis=1)
    
    # get places (airport codes)
    stations = pd.DataFrame.from_dict(rep_json['stations'])

    # Filter out legs where there is no itinary associated (so no price)
    if only_sellable:
        folders = folders[folders.is_sellable]
    
    # We merge to get both the premiere departure airport and the final airport
    folders = folders.merge(stations[['id', 'name', 'country','latitude','longitude']], left_on = 'departure_station_id',right_on = 'id', suffixes=['','_depart'] )
    folders = folders.merge(stations[['id', 'name', 'country','latitude','longitude']], left_on = 'arrival_station_id',right_on = 'id', suffixes=['','_arrival'] )
               
    # If no details asked we stay at leg granularity
    if not segment_details:
        return folders[['id', 'departure_date','arrival_date', 'nb_segments', 'name', 'country','latitude','longitude',
                       'name_arrival', 'country_arrival','latitude_arrival','longitude_arrival',
                       'cents', 'currency', 'comfort', 'flexibility', 'travel_class']].sort_values(by=['departure_date'])
    # else we break it down to each segment
    else : 
        # get segments (each unique actual flight)
        trips = pd.DataFrame.from_dict(rep_json['trips'])
        segments = pd.DataFrame.from_dict(rep_json['segments'])
        # Explode the list of segment associated to each leg to have one lie per segment
        folders_rich = pandas_explode(folders,'trip_ids')
        folders_rich = folders_rich.merge(trips[['id', 'segment_ids']], left_on = 'trip_ids',right_on = 'id', suffixes=['_global','_trip'] )
        folders_rich = pandas_explode(folders_rich,'segment_ids')
        folders_rich = folders_rich.merge(segments, left_on = 'segment_ids',right_on = 'id', suffixes=['','_seg'] )
        
        # Add relevant segment info to the exploded df (already containing all the leg and itinary infos)
        folders_rich = folders_rich.merge(stations[['id', 'name', 'country','latitude','longitude']], left_on = 'departure_station_id_seg',right_on = 'id', suffixes=['','_depart_seg'] )
        folders_rich = folders_rich.merge(stations[['id', 'name', 'country','latitude','longitude']], left_on = 'arrival_station_id_seg',right_on = 'id', suffixes=['','_arrival_seg'])

        # Recreate the order of the segment (not working so far)
        #folders_rich['seg_rank'] = folders_rich.groupby('id')["departure_date_seg"].rank("dense")
        # keep only the relevant information
        return folders_rich[['id', 'departure_date','arrival_date', 'nb_segments', 'name', 'country','latitude','longitude',
                       'name_arrival', 'country_arrival','latitude_arrival','longitude_arrival',
                       'cents', 'currency','departure_date_seg', 'name_depart_seg', 'country_depart_seg','latitude_depart_seg','longitude_depart_seg'
                        ,'arrival_date_seg','name_arrival_seg', 'country_arrival_seg','latitude_arrival_seg','longitude_arrival_seg',
                             'transportation_mean', 'carrier', 'train_name', 'train_number', 'co2_emission',
                             'flexibility', 'travel_class_seg']].sort_values(by=['departure_date'])


# function to get all trainline fares and trips 
def search_for_all_fares(date, origin_id, destination_id, passengers, include_bus = True, segment_details = False):
    # Define headers (according to github/trainline)
    headers = {
                'Accept': 'application/json',
                'User-Agent': 'CaptainTrain/43(4302) Android/4.4.2(19)',
                'Accept-Language': 'fr',
                'Content-Type': 'application/json; charset=UTF-8',
                'Host': 'www.trainline.eu',
            }

    session = requests.session()
    systems = ['sncf', 'db', 'idtgv', 'ouigo', 'trenitalia', 'ntv', 'hkx', 'renfe', 'cff', 'benerail', 'ocebo', 'westbahn', 'leoexpress', 'locomore', 'distribusion', 'cityairporttrain', 'obb', 'timetable']
    if include_bus:
        systems.append('busbud')
        systems.append('flixbus')

    data = {'local_currency': 'EUR'
            , 'search': {'passengers': passengers
                         , 'arrival_station_id': destination_id, 
                         'departure_date': date,
                         'departure_station_id': origin_id,
                         'systems': systems
                        }
           }
    post_data = json.dumps(data)

    tmp = dt.now()
    ret = session.post(url= "https://www.trainline.eu/api/v5_1/search",
                                        headers=headers,
                                        data=post_data)

    print(f'API call duration {dt.now() - tmp}')
    return format_trainline_response(ret.json(), segment_details=segment_details)

short_response = search_for_all_fares('2019-10-15T09:00:00+0200', 5009, 4917, passengers)
detail_response = search_for_all_fares('2019-10-15T09:00:00+0200', 5009, 4917, passengers, segment_details=True)


API call duration 0:00:02.402636
API call duration 0:00:01.929024


In [71]:
detail_response.head()

,id,departure_date,arrival_date,nb_segments,name,country,latitude,longitude,name_arrival,country_arrival,latitude_arrival,longitude_arrival,cents,currency,seg_rank,departure_date_seg,name_depart_seg,country_depart_seg,latitude_depart_seg,longitude_depart_seg,arrival_date_seg,name_arrival_seg,country_arrival_seg,latitude_arrival_seg,longitude_arrival_seg,transportation_mean,carrier,train_name,train_number,co2_emission,flexibility,travel_class_seg
0,142cde74e82f11e99c8468ba54f6b481,2019-10-15T08:08:00+02:00,2019-10-15T13:57:00+02:00,1,Le Puy-en-Velay,FR,45.042866,3.892421,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.83917,2.38278,7010,EUR,0,2019-10-15T08:08:00+02:00,Le Puy-en-Velay,FR,45.042866,3.892421,2019-10-15T10:15:00+02:00,Clermont-Ferrand,FR,45.778945,3.100543,train,sncf,TER,73704,3900.0,semiflexi,economy
36,142d0304e82f11e985c89ce67651055e,2019-10-15T08:08:00+02:00,2019-10-15T13:57:00+02:00,1,Le Puy-en-Velay,FR,45.042866,3.892421,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.83917,2.38278,11910,EUR,0,2019-10-15T10:25:00+02:00,Clermont-Ferrand,FR,45.778945,3.100543,2019-10-15T13:57:00+02:00,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.839170,2.382780,train,sncf,IC,5966,2400.0,flexi,first
35,142d25d2e82f11e98035b1ec6b39c4ef,2019-10-15T08:08:00+02:00,2019-10-15T13:57:00+02:00,1,Le Puy-en-Velay,FR,45.042866,3.892421,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.83917,2.38278,7600,EUR,0,2019-10-15T10:25:00+02:00,Clermont-Ferrand,FR,45.778945,3.100543,2019-10-15T13:57:00+02:00,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.839170,2.382780,train,sncf,IC,5966,2400.0,semiflexi,first
34,142ce036e82f11e993625ba334329e10,2019-10-15T08:08:00+02:00,2019-10-15T13:57:00+02:00,1,Le Puy-en-Velay,FR,45.042866,3.892421,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.83917,2.38278,7010,EUR,0,2019-10-15T10:25:00+02:00,Clermont-Ferrand,FR,45.778945,3.100543,2019-10-15T13:57:00+02:00,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.839170,2.382780,train,sncf,IC,5966,2400.0,semiflexi,economy
1,142d23e8e82f11e988e5ae187b0864fa,2019-10-15T08:08:00+02:00,2019-10-15T13:57:00+02:00,1,Le Puy-en-Velay,FR,45.042866,3.892421,Paris Bercy Bourgogne-Pays d’Auvergne,FR,48.83917,2.38278,7600,EUR,0,2019-10-15T08:08:00+02:00,Le Puy-en-Velay,FR,45.042866,3.892421,2019-10-15T10:15:00+02:00,Clermont-Ferrand,FR,45.778945,3.100543,train,sncf,TER,73704,3900.0,semiflexi,economy


# Exploration of json response

In [45]:
folders =  pd.DataFrame.from_dict(tmp['folders'])
print(folders.shape)
folders.sample(10)

(34, 32)


,arrival_date,arrival_different_from_requested,arrival_distance_from_requested,arrival_station_id,cents,comfort,currency,departure_date,departure_different_from_requested,departure_distance_from_requested,departure_station_id,digest,direction,flexibility,has_round_trip_fare,id,is_birthdate_required,is_birthplace_required,is_main_passenger_required,is_only_possible_travel_class,is_phone_number_mandatory,is_sellable,local_amount,local_currency,reference_folder_ids,refresh_hints,refreshable,required_identification_documents,search_id,system,travel_class,trip_ids
0,2019-10-15T13:57:00+02:00,False,NaN,4917,7010,high,EUR,2019-10-15T08:08:00+02:00,False,NaN,4683,c2cba80e4d51100995f5bb0bd672e4288c290418,outward,semiflexi,False,28c4a244e82b11e98076a4658bfb5306,True,False,False,False,False,True,"{'subunit': 7010, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,economy,[28c4a136e82b11e994a233f943a76442]
10,2019-10-15T16:57:00+02:00,False,NaN,4917,7390,medium,EUR,2019-10-15T09:51:00+02:00,True,1900.0,17079,729b53a435a385122b6e53d094d8936bd6d9372f,outward,semiflexi,False,28c58394e82b11e99b15b7321f74cc01,True,False,False,False,False,True,"{'subunit': 7390, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,economy,[28c582aee82b11e98f47c735b78a3dae]
11,2019-10-15T16:57:00+02:00,False,NaN,4917,8100,medium,EUR,2019-10-15T09:51:00+02:00,True,1900.0,17079,729b53a435a385122b6e53d094d8936bd6d9372f,outward,semiflexi,False,28c5cc82e82b11e990bd7a6d74f1d72c,True,False,False,False,False,True,"{'subunit': 8100, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,first,[28c5cb7ee82b11e995a44266d17bd2d6]
21,2019-10-15T22:57:00+02:00,False,NaN,4917,5210,high,EUR,2019-10-15T16:06:00+02:00,False,NaN,4683,ab32036f155454fa0849a588c146dcc633390b30,outward,semiflexi,False,2921cc6ce82b11e99c48d229e5b87a82,True,False,False,False,False,True,"{'subunit': 5210, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,first,[2921cb90e82b11e986ad2436743cd693]
13,2019-10-15T19:57:00+02:00,False,NaN,4917,7010,high,EUR,2019-10-15T12:18:00+02:00,False,NaN,4683,16c04f26bc02b557a1b2b5891924c72a1be221b3,outward,semiflexi,False,28c606d4e82b11e98392edf757512b8a,True,False,False,False,False,True,"{'subunit': 7010, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,economy,[28c605f8e82b11e98dbc3701d09aadd4]
17,2019-10-15T18:11:00+02:00,True,900.0,4924,16670,medium,EUR,2019-10-15T12:37:00+02:00,False,NaN,4683,911c7d3be5ac0a9b306c343fb163ac8c884f399d,outward,flexi,False,29215b38e82b11e999f81fe715db8eba,True,False,False,False,False,True,"{'subunit': 16670, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,first,[29215a70e82b11e98b085695adc53370]
25,2019-10-15T21:01:00+02:00,True,900.0,4924,17440,high,EUR,2019-10-15T16:19:00+02:00,False,NaN,4683,700a5253701da246b61a7cdac8ec558362d67b35,outward,flexi,False,292209d4e82b11e98631355a6d6eab4d,True,False,False,False,False,True,"{'subunit': 17440, 'subunit_to_unit': 100}",EUR,[],None,None,None,504991858,pao_sncf,first,[292208e4e82b11e99581de3468f48bbd]
9,2019-10-15T20:40:00+02:00,True,12000.0,2ed6a652c776e5fd88847b65f6cbe670,3990,low,EUR,2019-10-15T09:30:00+02:00,False,NaN,4683,005a55e668eee02feffe405b304fb691cb76b640,outward,nonflexi,False,292c59cae82b11e9933d85302a6e2250,True,False,False,False,False,True,"{'subunit': 3990, 'subunit_to_unit': 100}",EUR,"[289c41aae82b11e99d86d8418291ee58, 289a6f9ce82...",[],none,[],504991858,capitaine train,first,[292c5ac4e82b11e999325f6b844b2d9a]
33,2019-10-16T11:08:00+02:00,True,29500.0,4757,3620,low,EUR,2019-10-15T17:35:00+02:00,False,NaN,4683,1a8ee3ccf312ea0b39f9b6ed002cb20c1ff9e905,outward,nonflexi,False,292bec56e82b11e99a0506995a144145,True,False,False,False,True,True,"{'subunit': 3620, 'subunit_to_unit': 100}",EUR,"[289cbd1ae82b11e99d071fe57ebf6f57, 28a90412e82...",[],none,[],504991858,capitaine train,economy,[292bee5ee82b11e98eff6f3a72bec3c8]
28,2019-10-15T22:57:00+02:00,False,NaN,4917,11880,medium,EUR,2019-10-15T

In [46]:
trips =  pd.DataFrame.from_dict(tmp['trips'])
print(trips.shape)
trips.head()

(34, 13)


,arrival_date,arrival_station_id,cents,currency,departure_date,departure_station_id,digest,folder_id,id,local_amount,local_currency,passenger_id,segment_ids
0,2019-10-15T13:57:00+02:00,4917,7010,EUR,2019-10-15T08:08:00+02:00,4683,c2cba80e4d51100995f5bb0bd672e4288c290418,28c4a244e82b11e98076a4658bfb5306,28c4a136e82b11e994a233f943a76442,"{'subunit': 7010, 'subunit_to_unit': 100}",EUR,3c29a998-270e-416b-83f0-936b606638da,"[28c49d94e82b11e988c12cecada7232e, 28c49f24e82..."
1,2019-10-15T13:57:00+02:00,4917,7600,EUR,2019-10-15T08:08:00+02:00,4683,c2cba80e4d51100995f5bb0bd672e4288c290418,28c4846ce82b11e99210aefe62d05342,28c48386e82b11e982b46602b77c317a,"{'subunit': 7600, 'subunit_to_unit': 100}",EUR,3c29a998-270e-416b-83f0-936b606638da,"[28c48066e82b11e999d693da8de1bff7, 28c481bae82..."
2,2019-10-15T13:57:00+02:00,4917,11910,EUR,2019-10-15T08:08:00+02:00,4683,c2cba80e4d51100995f5bb0bd672e4288c290418,28c4672ae82b11e991eeb7a67d489029,28c4661ce82b11e99abe8fc1fb880bc8,"{'subunit': 11910, 'subunit_to_unit': 100}",EUR,3c29a998-270e-416b-83f0-936b606638da,"[28c462cae82b11e981ff60550b8fe993, 28c46450e82..."
3,2019-10-15T13:31:00+02:00,4924,7800,EUR,2019-10-15T08:42:00+02:00,4683,beee313de46e00c51f5d035e110a4125b7445d6e,28c520a2e82b11e98f32d9ca648e8e74,28c51fb2e82b11e98b36064335c5f688,"{'subunit': 7800, 'subunit_to_unit': 100}",EUR,3c29a998-270e-416b-83f0-936b606638da,"[28c515a8e82b11e9847c61ec0372a8af, 28c51738e82..."
4,2019-10-15T13:31:00+02:00,4924,8670,EUR,2019-10-15T08:42:00+02:00,4683,beee313de46e00c51f5d035e110a4125b7445d6e,28c4edb2e82b11e99c1ba19f645a9323,28c4eccce82b11e9863025a8412d5a1d,"{'subunit': 8670, 'subunit_to_unit': 100}",EUR,3c29a998-270e-416b-83f0-936b606638da,"[28c4e89ee82b11e9897918a75c5e04ee, 28c4ea74e82..."


In [158]:
stations =  pd.DataFrame.from_dict(tmp['stations'])
print(stations.shape)
stations.head()

(11, 10)


,address,country,id,info,latitude,longitude,name,parent_name,parent_slug,slug
0,NaN,FR,4594,None,47.926801,1.906629,Les Aubrais,Orléans,orleans,les-aubrais
1,NaN,FR,4921,None,48.842285,2.364891,Paris Austerlitz,Paris,paris,paris-austerlitz
2,NaN,FR,192,None,47.907960,1.904666,Orléans Centre,Orléans,orleans,orleans-centre
3,"[Avenue Georges Pompidou, 45380 La Chapelle Sa...",FR,5f8b2f88aaaea92b5e89f9ccb66fdfd5,None,47.897321,1.854379,Orléans Pompidou,Orléans,orleans,orleans
4,"[210 Quai de Bercy, 75012 Paris]",FR,81955b9e0ea9153c55fdda1874a0d0ba,None,48.835311,2.380519,Paris - Bercy Seine,Paris,paris,paris


In [156]:
comfort_classes =  pd.DataFrame.from_dict(tmp['comfort_classes'])
print(comfort_classes.shape)
comfort_classes.head()

(23, 15)


,always_display,cents,condition_id,currency,description,id,is_available,is_default,local_amount,local_currency,name,options,reservation,segment_id,title
0,False,0,710205aee76f11e98d32ea48d5eba269,EUR,Un siège standard.,7102082ee76f11e98408c76d1b046ba4,True,True,"{'subunit': 0, 'subunit_to_unit': 100}",EUR,pao.default,"{'seats': [], 'extras': [{'title': 'Espace vél...",none,6f398bc0e76f11e98fce89915e349de9,Normal
1,False,0,710205aee76f11e98d32ea48d5eba269,EUR,Un siège standard.,71020d24e76f11e9877cfd33ad4d7556,True,True,"{'subunit': 0, 'subunit_to_unit': 100}",EUR,pao.default,"{'seats': [], 'extras': [{'title': 'Espace vél...",none,6f39a70ee76f11e98808b8735cddc48a,Normal
2,False,0,710205aee76f11e98d32ea48d5eba269,EUR,Un siège standard.,710211ace76f11e99db362d00c622d5b,True,True,"{'subunit': 0, 'subunit_to_unit': 100}",EUR,pao.default,"{'seats': [], 'extras': [{'title': 'Espace vél...",none,6f39c0b8e76f11e98d9ea071abe561cc,Normal
3,False,0,710205aee76f11e98d32ea48d5eba269,EUR,Un siège standard.,71021652e76f11e98b344830bff4560f,True,True,"{'subunit': 0, 'subunit_to_unit': 100}",EUR,pao.default,"{'seats': [], 'extras': [{'title': 'Espace vél...",none,6f39da9ee76f11e984ec44ea14608cb4,Normal
4,False,0,71021a76e76f11e98352ca21036d3572,EUR,None,71021b8ee76f11e99208b4656c0bb013,True,True,"{'subunit': 0, 'subunit_to_unit': 100}",EUR,idbus.regular,"{'seats': None, 'extras': [{'title': 'WiFi', '...",None,70fe5c60e76f11e993afc4edb096b779,None


In [157]:
conditions =  pd.DataFrame.from_dict(tmp['conditions'])
print(conditions.shape)
conditions.head()

(8, 7)


,digest,fare_code,id,is_independent,name,segment_id,short_description
0,d9f8ff8d1e7d797f81656e1d6102e0de,NU94,710205aee76f11e98d32ea48d5eba269,True,Prix Flash,6f398bc0e76f11e98fce89915e349de9,Valable uniquement dans\nle train désigné sur ...
1,408b5f6c22e8d97983cdea4af99ffa59,Standard,71021a76e76f11e98352ca21036d3572,True,BlaBlaBus,70fe5c60e76f11e993afc4edb096b779,Non remboursable. Non échangeable. Présence à ...
2,80c77b6465f6b5ca3e1a88e074686871,FA11,710231d2e76f11e995bedc41db10b829,True,BUSINESS PREMIERE,6f3a6248e76f11e99deebb831c8d5514,Billet échangeable sans frais jusqu'à 30 minut...
3,fd18f8a885eb0dea09a82041259b4522,JR11,71023a06e76f11e99079344a7fa08605,True,PREMIERE,6f3a8610e76f11e9944fd9a331a05cd3,Billet échangeable et remboursable sans frais ...
4,4dd74e357ed3a9f0564c74d570075ac0,JR11,71023d3ae76f11e9914fa26b5b1c6a48,True,PREMIERE,6f3a880ee76f11e996b3b26295d3068e,Billet échangeable et remboursable avec retenu...
